### importing require packages

In [3]:
from __future__ import print_function

import json
import os
import numpy as np
import sys
import h5py

from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from keras.engine import Input
from keras.layers import Embedding, merge
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.preprocessing import sequence
from embeddings import Embeddings
from keras.callbacks import ModelCheckpoint

from nltk.tokenize import word_tokenize

## Instantiate Embeddings 

In [4]:
embeddings = Embeddings(100, 3, 1, 4)

### getting data from preprocessing

In [5]:
word2vec_weights = embeddings.get_weights()
word2index, index2word = embeddings.get_vocabulary()
word2vec_model = embeddings.get_model()
tokenized_indexed_sentences = embeddings.get_tokenized_indexed_sentences()

### generating training data

In [6]:
window_size = 3
vocab_size = len(word2index)
print(vocab_size)
#sorted(window_size,reverse=True)
#sentence_max_length = max([len(sentence) for sentence in tokenized_indexed_sentence ])

136086


In [7]:
seq_in = []
seq_out = []
# generating dataset
for sentence in tokenized_indexed_sentences:
    for i in range(len(sentence)-window_size-1):
        x = sentence[i:i + window_size]
        y = sentence[i + window_size]
        seq_in.append(x)#[]
        seq_out.append(word2vec_weights[y])

# converting seq_in and seq_out into numpy array
seq_in = np.array(seq_in)
seq_out = np.array(seq_out)
n_samples = len(seq_in)
print ("Number of samples : ", n_samples)

Number of samples :  3365222


## Defining model

In [8]:
# Changes to the model to be done here
model = Sequential()
model.add(Embedding(input_dim=word2vec_weights.shape[0], output_dim=word2vec_weights.shape[1], weights=[word2vec_weights]))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512))
model.add(Dropout(0.2))
model.add(Dense(word2vec_weights.shape[1], activation='sigmoid'))
model.compile(loss='mse', optimizer='adam',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         13608600  
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 512)         1255424   
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 512)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               51300     
Total params: 17,014,524
Trainable params: 17,014,524
Non-trainable params: 0
________________________________________________________________

In [9]:
model_weights_path = "../weights/LSTM_1_Layer"
if not os.path.exists(model_weights_path):
    os.makedirs(model_weights_path)
checkpoint_path = model_weights_path + '/weights.{epoch:02d}-{val_acc:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath=checkpoint_path, monitor='val_acc', verbose=1, save_best_only=False, mode='max')

## Train Model

In [10]:
model.fit(seq_in[:1000], seq_out[:1000], epochs=1, verbose=1, validation_split=0.2, batch_size=32, callbacks=[checkpoint])

Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 28s - loss: 0.8781 - acc: 0.1288 - val_loss: 0.5489 - val_acc: 0.1750


### model predict

In [12]:
start = 99
pattern = list(seq_in[start])
print("\"",' '.join(index2word[index] for index in pattern))
for i in range(10):
    prediction = model.predict(np.array([pattern]))
    pred_word = word2vec_model.similar_by_vector(prediction[0])[0][0]
    sys.stdout.write(pred_word)
    pattern.append(word2index[pred_word])
    pattern = pattern[1:len(pattern)]

" as Super Bowl
successfulsuccessfulsuccessfulsuccessfulsuccessfulsuccessfulsuccessfulsuccessfulsuccessfulsuccessful

## Accuracy

In [1]:
def accuracy(no_of_preds):
    correct = 0
    count = 0
    for i in range(no_of_preds):
        start = i
        sentence = list(x[start])
        prediction = model.predict(np.array([sentence]))
        pred_word = w2v_model.similar_by_vector(prediction[0])[0][0]
        next_word_index = list(x[start+1])
        next_word = idx2word[next_word_index[0]]
        sim = w2v_model.similarity(pred_word,next_word)
        if (sim >= 0.7):
            correct+=1
        count+=1
            
        
    print('correct: '+str(correct)+(' wrong: ')+str(wrong))
    accur = float(correct/(c))
    print('accuracy = ',float(accur))
    

In [2]:
accuracy(100)

NameError: name 'x' is not defined